# Map Reduce  

**Map Reduce** è un framework inventato e brevettato da Google per supportare la computazione distribuita su una grande mole di dati, nasce dall'ispirazione delle funzioni **map** e **reduce** del functional programming.

Il processo si divide in due steps:
- MAPPING: L'input viene diviso in sotto parti finchè non raggiunge la dimensione minima processuale, restituisce un risultato

- REDUCE: Ogni valore derivato dal processo di mapping viene processato effettuando una **join** di tutti i valori in un singolo risultato aggregato.


<img src="files/MapReduce.jpeg" width="640"/>



# Why Map Reduce

Map Reduce in 140 Caratteri (From Twitter):

* Some computing tasks can be split up into small chunks, processed simultaneously, and then reassembled at the end

* MapReduce is the algorithm google uses to allow many machines to break a problem into smaller chunks and process data.

* Allows you to quickly process very large amounts of data very quickly using distributed computing, often using cheap hardware
 
* Map reduce is important because it allows for problems to be distributed rather easily.
 
* Parallelism and multi-core
 
* Problem hard. Many computers work on problem. Chop problem up (map). Merge results (reduce). Works for cloud or massive multi-core.
 
 
## Concetti Chiave

1) **Parallelismo & multicore**: MapReduce è ottimo per lavorare con grandi dataset su un cluster distribuito grazie alle potenzialità del framework, in grado di spezzare un grande problema in tanti piccoli sotto-problemi, effettuando un **merge** di tutti i risultati come step finale.
 
2) **Commodity-hardware**: MapReduce può funzionare con diversi linguaggi di programmazione senza nessun requisito hardware particolare. Per questa ragione MapReduce è considerata una soluzione a basso costo.


# Map Reduce Examples

MapReduce diventa realmente rilavante nel momento in cui un singolo computer non è più sufficiente per garantire l'eleborazione del dataset a causa dell'elevato volume o complessità. Questo è il motivo per cui molti vendor di soluzione Cloud hanno integrato soluzione MapReduce nel loro catalogo di servizi.

Vedremo successivamente come utilizzare sistemi in Cloud per l'elaborazione di dati distribuita su molteplici macchine (nel caso pratico utilizzeremo AWS - Amazon Web Services). Il fatto di essere *cloud-ready* è un fattore molto importante nel momento in cui si comincerà a lavorare sui dati, motivo per cui il paradigma MapReduce è diventato cosi diffuso.

Siccome MapReduce permette di utilizzare diversi linguaggi / framework, andremo a utilizzare librerie Python e MongoDB (il quale implementa nativamente logiche di MapReduce) e vedremo un introduzione a Hadoop e una demo di Spark.

Esempi di utilizzo:

- WordCount
- Rimozione di duplicati all'interno del dataset (data-cleaning)
- trasposizione di matrici
- Followers dei followers
- Calcolo di statistiche
- Recommendation systems
- ...

## Conclusione

MapReduce è quindi un tool che potremmo usare durante la presentazione o l'esplorazione del dataset. A noi, come tanti data scientist, piace perchè:

- Ottimo strumento per lavorare sui dati in rapidamente
- Ci permette di scalare facilmente orizzontalmente
- Non ci serve sapere tutti i segreti che sono dietro a MapReduce, lo usiamo, funziona, è ben documentato ed è quindi, per noi, più che sufficiente :)


## Resources

Qui --> [LINK](http://had00b.blogspot.co.uk/2013/08/mapreduce-gentle-introduction.html) potete trovare un introduzione più strutturata a MapReduce!

Qui --> [LINK](https://gigaom.com/2013/03/04/the-history-of-hadoop-from-4-nodes-to-the-future-of-data/) potete trovare una introduzione su Hadoop (vedi dopo per maggiori informazioni)

Per maggiori esempi o informazioni, trovate esempi, documentazione e tutorial [QUI](https://github.com/alexcomu/hadoop-mapreduce)

## Practice Example - Word Count

<img src="files/MapReduce_example.png"/>

## Esempio senza l'utilizzo di MapReduce

L'obiettivo del problema è di calcolare la somma del doppio di una serie di numeri.

Quindi, se abbiamo come input **[1, 2, 3]** il risultato sarà: **2+4+6 -> 12**


In [1]:
# Genero una lista lunga 1000
numbers = range(1000)

def doubled_sum(values):
    total = 0 # inizializzo una variabile per la somma
    for n in values:
        # per ogni numero calcolo il doppio e effettuo la somma
        total += n*2
    # Una volta terminato il ciclo, restituisco il valore
    return total

print doubled_sum(numbers)

999000


## Esempio utilizzando MapReduce

Siccome Python supporta il paradigma del **functional programmin**, si ha a disposizione tutto il necessario per implementare il paradigma **map-reduce** direttamente in linguaggio python nativo.

Entrambi gli step possono essere eseguiti più volte (L'output del reducer può diventare l'input di un ulteriore mapper e cosi via).

Ecco gli step da seguire per risolvere il problema:

### Mapper

Ha il compito di effettuare il mapping di ogni singolo input, restituendo come output il suo valore moltiplicato per 2

### Reducer

Ha il compito di effettuare la somma degli output multipli del mapper.


In [13]:
numbers = range(1000)

def mapper(value):
    return value*2

def reducer(*args):
    # print args
    return sum(args)

#map_result = map(mapper, numbers)
##print map_result
#result = reduce(reducer, map_result)

result = reduce(reducer, map(mapper, numbers))
print result

999000


Il precedente esempio in "pure python" funziona ma ovviamente non efficiente siccome non permette una delle funzionalità core di MapReduce: **working parallely**.

Si può facilmente dedurre che ogni **mapper** e **reducer** lavorano su un subset dei dati e sono completamente indipendenti dallo status degli altri mapper e reducer. Quindi il processo può procedere in parallelo senza intoppi.

## Parallel Map Reduce in Pure Python

Ecco un esempio di simulazione di un processo map reduce in parallelo in python utilizzando il modulo **multiprocessing**.

In [42]:
from itertools import islice
import multiprocessing

class ParallelMapReduce(object):
    def __init__(self, map_func, reduce_func, num_workers=None):
        self.num_workers = num_workers
        self.map_func = map_func
        self.reduce_func = reduce_func
        self.pool = multiprocessing.Pool(num_workers)
    
    def partition(self, n, iterable):
        i = iter(iterable)
        piece = list(islice(i, n))
        # print "partizione 1: ", piece
        while piece:
            # print "--> ", piece
            yield piece
            piece = list(islice(i, n))
    
    def __call__(self, inputs):
        values = self.pool.map(self.map_func, inputs)
        
        print '>>> MAPPED VALUES (%s values): %s, ...' % (len(values), str(values[:10]))

        values = self.pool.map(self.reduce_func, 
                               self.partition(len(values)//self.num_workers, values))
        print '>>> REDUCED VALUES', values

        return self.reduce_func(values)

L'esempio crea **N** partizioni in base a quanti **N** workers sono stati dichiarati, a questo punto ogni worker riceve i vari input e restituisce il risultato di ogni singolo step.

In [50]:
numbers = range(1000)

def mapper(value):
    return value*2

def reducer(values):
    return sum(values)

mapreduce = ParallelMapReduce(mapper, reducer, 10)
print ">>> RESULT: %s" % mapreduce(numbers)

>>> MAPPED VALUES (1000 values): [0, 2, 4, 6, 8, 10, 12, 14, 16, 18], ...
>>> REDUCED VALUES [9900, 29900, 49900, 69900, 89900, 109900, 129900, 149900, 169900, 189900]
>>> RESULT: 999000


### File System

Un **file system** (abbreviazione: FS) indica informalmente un meccanismo con il quale i file sono posizionati e organizzati o su un dispositivo di archiviazione o su una memoria di massa, come un disco rigido o un CD-ROM e, in casi eccezionali, anche sulla RAM.

[Definizione su WIKIPEDIA](https://it.wikipedia.org/wiki/File_system)

### File System Distribuito

Un **file system distribuito** (in inglese, Distributed File System, o DFS) è un particolare file system che permette la memorizzazione di files e risorse in dispositivi di archiviazione distribuiti in una rete informatica, anziché letti o archiviati in maniera centralizzata su di un dispositivo locale, e resi dunque disponibili attraverso un meccanismo client-server tra dispositivi remoti.

[Definizione su WIKIPEDIA](https://it.wikipedia.org/wiki/File_system_distribuito)

## Distributed Map Reduce

Il nostro esempio di map reduce lavora in parallelo usando i core multipli del nostro computer, non è però in grado di distribuire il lavoro su computer differenti. Siccome MapReduce nasce per poter eseguire lo stesso processo in un ambiente distribuito è di vitale importanza riuscire a creare un codice che lavori attraverso molteplici server in parallelo.

Questa è la ragione per cui sono stati creati differenti framework per MapReduce, ecco alcune soluzioni per python:


### Disco ###

Progetto nato al centro di ricerca Nokia, è una delle soluzioni con un ottimo tradeof fra complessità e funzionalità. Python è considerato il linguaggio standard per implementare logiche mapReduce attraverso i workers messi a  disposizione dal **job dispatcher**.

Fornisce inoltre un suo Distributed File System (DDFS) e Distributed Database (DiscoDB) che può essere utilizzato per salvare dati attraverso i vari workers.

### OctoPy ###

Soluzione "pure Python" semplice e rapida da implementare ma non fornisce disco condiviso per i dati. Può essere una buona soluzione per problemi di piccola dimensione dove il costo della configurazione del cluster ridotto può essere un buon benefit.

### Hadoop ###

Hadoop è lo standard de facto in frameworks MapReduce, implementato in Java rappresenta la soluzione più complessa per setup e mantenimento. Tramite l'**Hadoop Streaming** è possibile eseguire mappers e reducers in qualsiasi linguaggio di programmazione che può essere chiamato da uno script shell. Hadoop Streaming è usato da diverse librerie Python che permettono l'implementazione di soluzioni MapReduce in facilità.

Come Disco fornisce una soluzione a 360 gradi fornendo un Distributed File System **HDFS** dove i dati possono essere salvati durante l'esecuzione degli script.


# Amazon EMR - Elastic MapReduce


EMR, **Elastic Map Reduce** è la soluzione Amazon per cluster Hadoop in cloud.

Creare e gestire un cluster Hadoop è un processo lungo e complesso, è buona norma utilizzare servizi di cloud computing on demand. Durante le lezioni vedremo come utilizzare script MapReduce in locale, ma vedremo anche come utilizzare ERM per eseguire gli stessi script. Noteremo che EMR ha alti costi di kickstart, di conseguenza andrà utilizzato solo quando la mole di dati è molto elevata.

Il tutto funziona su Amazon **Elastic Compute Cloud** (EC2) per la creazione di instanze ad hoc collegate fra di loro per la creazione del cluster Hadoop. Sfrutta inoltre **Simple Storage Service** (S3) per la gestione di file system condiviso, ogni nodo potrà accedere e scaricare / uploadare dati.

<img src="files/ec2.png" width="640"/>

### Costo di Setup (Kickstart cost)

Il problema nell'utilizzo di EMR è nell'elevato costo di avvio / setup causato da:

 - Le istanza EC2 necessitano di essere create e accese per cominciare a processare dati
 - I dati necessitano di essere caricati su un bucket di S3 per permettere alle istanze di accedervi e di iniziare il lavoro
 
Per queste ragioni, siccome l'avvio necessita di attendere minuti, sarà più comodo in fase di esercizio / learning eseguire gli script in locale tramite un framework multiprocess che simula i nodi Hadoop. Vedremo però come sarà possibile utilizzare gli stessi script senza la necessità di effettuare modifiche per eseguirli in ambiente distribuito direttamente su Amazon EMR.


# Hadoop Streaming

La funzionalità che permette di eseguire algoritmi MapReduce in un qualsiasi linguaggio su Hadoop è chiamata Hadoop Streaming, è un tool che riceve come parametro di input un qualsiasi software eseguibile da linea di comando costituito da un mapper e un reducer.

Questo rende possibile l'utilizzo di uno script Python all'interno del cluster Hadoop che dovrà utilizzare il protocollo per la gestione di input e output (i dati saranno separati da un "\n"):

 - Ogni "linea" di input inviata al mapper si deve considerare come un singolo valore da mappare
 - Ogni "linea" di output restituita dal mapper è da considerare come una tupla (key, value) dove key e value sono rispettivamente reparate da un TAB ("\t")
 
Ecco un esempio di classico I/O di uno Streaming Hadoop:

`key1\tvalue1\nkey2\tvalue2\n`

#  DEMO

## Trovare film simili sulla base del Rating degli utenti - Movie recommendation

Utilizzando un dataset di voti raccolti da utenti anonimi (fonte del dataset: MovieLands), seguiamo i seguenti step:

- Troviamo tutte le coppie di film visti dalle stesse persone
- Calcoliamo la *similarity* basato sul rating degli utenti che hanno visto entrambi
- Ordiniamo per film utilizzando il livello di *similarity*
    

### MapReduce Problem

MapReduce problem, 3 steps (tempo di calcolo circa --> XX per dataset da 100K entries):

    Mapper: Estraggo tutti gli utenti e i film visti con associato il rating dato -> (Movie, Rating)
    Reducer: Raggruppo per utente tutti i film visti con associato per ognuno il rating

    Mapper: Restituisco tutti i film visti da ogni utente con il rating (movie1, movie2) ->  (rating1, rating2)
    Reducer: Calcolo la rating-based similarity per ogni coppia di film (movie1, movie2) -> (similarity, numero di utenti che ha visto entrambi)

    Mapper: Recupero il nome del film e la sua similarity
    Reducer: Restituisco l'output
    
    Output: -> Lista di film in ordine per nome con lista di film correlati in base al grado di similarity
    Format:  "MOVIE_NAME" ["SIMILAR_MOVIE", "SIMILAR_POINT (From 0 to 1)", "Number of people who votes both"]
    Example: "Start Wars 1977" ["Empire Strikes Back 1980", 0.989552207, 345]

### Risultato su Star Wars 1977

    GRADO DI SIMILARITY [ FILM_NAME, Numero di voti da parte di utenti]
    0.9895522078385338	["Empire Strikes Back, The (1980)", 345]
    0.9857230861253026	["Return of the Jedi (1983)", 480]
    0.9817600988726190	["Raiders of the Lost Ark (1981)", 380]    
    0.9735394829992481	["Indiana Jones and the Last Crusade (1989)", 304]    
    0.9672204017083984	["Toy Story (1995)", 381]    
    0.9672159797078707	["Back to the Future (1985)", 309]
    0.9669998681287573	["Star Trek: First Contact (1996)", 316]
    0.9647806324397128	["Godfather, The (1972)", 357]
    0.9644359422817009	["Silence of the Lambs, The (1991)", 335]
    0.9595898029305344	["Contact (1997)", 334]
    0.9568016154365081	["Twelve Monkeys (1995)", 324]    
    0.9546838657901240	["Fargo (1996)", 394]
    0.9522886451418612	["Rock, The (1996)", 312]
    


### Come eseguire la DEMO

Prima di tutto assicurarsi di avere MrJob installato (verrà spiegato nel capitolo 5).

Sarà sufficiente eseguire lo script contenuto nel file **script.py**:

    python script.py --items=u.item u.data > result_file
    
Dopo un paio di minuti (la velocità dipende dal vostro PC) il risultato verrà stampato nel file **result_file**.

Successivamente si può applicare il secondo script per filtrare sul file dei risultati:

    python clean.py result_file
    
Per cercare uno specifico film sarà sufficiente cambiare il nome del film ricercato nel file **clean.py** in riga 8.

